In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [ ]:
df = pd.read_csv("data/jobs_in_data.csv")

canada = df[df["employee_residence"] == "Canada"]
us = df[df["employee_residence"] == "United States"]
uk = df[df["employee_residence"] == "United Kingdom"]
# df = df.astype({'salary_in_usd':'int'})
# df.dropna
df

In [ ]:
# corr = df.corr(numeric_only=False)
# corr

In [ ]:
def order(salary_group):
    # salary_group = df.groupby(["experience_level", "work_setting"])
    # Get the groups manually by experience level
    junior_group = salary_group.get_group('Entry-level')
    mid_level_group = salary_group.get_group('Mid-Level')
    senior_group = salary_group.get_group('Senior')
    exec_group = salary_group.get_group('Executive')

    # Concatenate the groups in the desired order
    ordered_group = pd.concat([junior_group, mid_level_group, senior_group, exec_group])
    return ordered_group

In [ ]:
# exp = df[["experience_level", "work_setting"]].value_counts()
# exp = df[["experience_level", "work_setting"]]["salary_in_usd"].mean()
# print(exp)
# can't directly perform something like mean() on the this dataframe, have to use groupby()

pd.set_option('display.max_columns', None)
# print(job_type)

# tried to order the items by one of the categories, works for groupby object but not a dataframe?

# order_mapping = {"Entry-level": 0, "Mid-level": 1, "Senior": 2, "Executive": 3}
# salary_group = df.sort_values("experience_level", key=lambda x: x.map(order_mapping))
order_mapping = ["Entry-level", "Mid-level", "Senior", "Executive"]
# df = df.reindex(order_mapping, level = 7)
# df['experience_level'] = df['experience_level'].map(order_mapping)
salary_group = df.groupby(["experience_level", "work_setting"]) #.apply(lambda x: x.sort_values(by='experience_level'))
# salary_group = salary_group.swaplevel().sort_index()
# salary_group = salary_group.apply(lambda x: x.reindex(order_mapping))
# salary_group = salary_group.reset_index(level = 0)
# salary_group = salary_group.reindex(order_mapping, level=0)
# salary_group = order(salary_group)
print(salary_group["salary_in_usd"])
print(salary_group["salary_in_usd"].median())

unique_combo =  list(salary_group.groups.keys())
# setting = df["work_setting"].unique()
# print(setting)
# print(len(unique_combo))

# creating a new column that combines the 2 categories, don't really like this approach

# df["combined"] = df['experience_level'] + ', ' + df['work_setting']
# print(df["combined"])

#### Plotting them as is to see some distribution

In [ ]:

fig, ax = plt.subplots(figsize=(10,5)) 
number_of_bins = 50
salary = df["salary_in_usd"]
h = ax.hist(salary, bins=number_of_bins, edgecolor = "red")
ax.set_ylabel("Number of jobs")
ax.set_xlabel("Salary(USD)")
# ax.set_yscale("log")

In [ ]:
df.boxplot(column="salary_in_usd", by=["experience_level", "work_setting"], figsize=(12,6), grid=False, vert = False)
plt.suptitle("")
plt.title("Salary Box Plot")
plt.tight_layout()
# plt.show()

#### Assign colours based on unique combinations

In [ ]:
from matplotlib import colormaps
colors = colormaps['viridis']

color_list = [colors(i) for i in np.linspace(0,1,len(unique_combo))]

color_dict = {str(unique_combo[i]):color_list[i] for i in range(len(unique_combo))}
print(color_dict)

# order_mapping = {"Entry-level": 1, "Mid-level": 2, "Senior": 3, "Executive": 4}
# # sorted_combo = sorted(unique_combo, key=lambda x: order_mapping[x[0]])
# # key_function = lambda x: (order_mapping[x[0]], order_mapping[x[1]])
# # sorted_salary = salary_group.sort_values(by="experience_level", key=lambda x: x.map(order_mapping))
# def custom_sort_key(index):
#     return tuple(order_mapping[level] for level in index)

# sorted_grouped = salary_group.apply(lambda x: x.sort_values(by=["experience_level"], key=custom_sort_key))

# print(sorted_grouped)

In [ ]:
rows = 3
cols = 4
fig, ax = plt.subplots(rows, cols, figsize=(20,16))

for num, i in enumerate(unique_combo):
    temp_salary = df[(df["experience_level"]== i[0]) & (df["work_setting"] == i[1])]["salary_in_usd"].values
    col = num%cols
    row = int(np.floor(num/cols))
    axis = ax[row,col]
    axis.plot(temp_salary,np.arange(len(temp_salary)), "o", color=color_list[num], ms = 2.5)
    axis.set_title(i)
    axis.tick_params(axis='x', labelrotation=25)
    # axis.set_xticklabels(axis.get_xticks(), ha = "right")
    axis.set_xlim((10000,500000))
    # not setting a boundary on y axis because there are too many more entries for senior positions
    # axis.set_ylim((0, 4500))

#### Central Tendency
##### Mean


In [ ]:
fig1, ax1 = plt.subplots(figsize=(10,6)) 
combo_label = [f"{i[0], i[1]}" for i in unique_combo]
mean_sal = salary_group["salary_in_usd"].mean()
ax1.barh(combo_label, mean_sal ,0.5,color=color_list)
ax1.set_xlabel("Salary(USD)")

# inverted so they are arranged from top to bottom
ax1.invert_yaxis()
ax1.set_title("Salary mean")
# ax1.tick_params(axis='x', labelrotation=90)

##### Range

In [ ]:
def range(x):
    return x.max() - x.min()

# salary_range = salary_group.apply(range).sort_values()

# reset index here is necessary because of the new column has no name
# really not sure why reset_index() assigns a 0 to the salary despite the range() method calls for the salary column, but hey it works
# even weirder is that its not a string "0", but literally the number 0?
# apparently this can be fixed by calling salary_group["salary_in_usd"].apply, rather than salary_group.apply()
salary_range = salary_group["salary_in_usd"].apply(range).reset_index()#.sort_values(by="salary_in_usd")
# salary_test = salary_group.apply(range)
# print(salary_test)
# combined_group = df.groupby(["combined"])
# salary_range = combined_group.apply(range)
# print(salary_range)

In [ ]:
# print(salary_group["salary_in_usd"])

fig, ax = plt.subplots(figsize=(12,5))
ax.set_ylabel("salary range")

# using combo_label directly as x axis means I can't change the horizontal alightment

ax.bar(salary_range.index, salary_range["salary_in_usd"], 0.5, color =color_list)
# ax.tick_params(axis='x', labelrotation=35)
ax.set_xticks(salary_range.index)
ax.set_xticklabels(unique_combo, rotation=45, ha='right')

In [ ]:
test = df["salary_in_usd"].dtype
print(test)

#### Standard Deviation

In [ ]:
std_salary = salary_group["salary_in_usd"].std()
print(std_salary)
std_salary = std_salary.reset_index()
print(std_salary)

fig, ax = plt.subplots(figsize=(12,5))
ax.set_ylabel("Salary(USD)")
ax.set_title("Standard deviation of salary")


ax.bar(std_salary.index, std_salary["salary_in_usd"], 0.5, color = color_list)
ax.set_xticks(salary_range.index)
ax.set_xticklabels(unique_combo, rotation=45, ha='right')

#### Z-Score

In [ ]:
def calc_z(x):
    return (x - x.mean()) / x.std()

In [ ]:

z_score = salary_group["salary_in_usd"].apply(calc_z).reset_index()
# z_score = z_score.reset_index()
print(z_score)

fig, ax = plt.subplots(figsize = (12,5))
ax.plot(z_score["salary_in_usd"], "x")
ax.set_title("Z-score")


In [ ]:
# grouped = df.groupby("label")
sem_salary = salary_group["salary_in_usd"].sem().reset_index()
# print(sem_salary)

fig, ax = plt.subplots(figsize=(12,5))
ax.set_ylabel("Salary(USD)")
ax.set_title("Salary standard error of mean")
print(sem_salary.values)
ax.bar(sem_salary.index, sem_salary["salary_in_usd"], 0.5, color = color_list)
ax.set_xticks(salary_range.index)
ax.set_xticklabels(unique_combo, rotation=45, ha='right')